In [5]:
using Pkg
Pkg.activate("./")
# Pkg.instantiate()

  Activating project at `~/inverseDesign`


In [6]:
using GLMakie 
using StaticArrays: SVector
using LinearAlgebra
using LinearSolve
using Meshes
using BenchmarkTools

In [9]:
include("./src/GeometryUtils.jl")
include("./src/BoundaryWall.jl")

┌ Warning: Replacing docs for `Main.GeometryUtils.buildAsymmetricResonator :: Tuple{Float64, Float64}` in module `Main.GeometryUtils`
└ @ Base.Docs docs/Docs.jl:243


divideMeshBoundary

In [10]:
HBAR = 1.0
MASS = HBAR/2
SIGMA= (2*MASS/HBAR^2)*(1/4*im)
N    = 400
NDOM = 150
ϕ    = -90
K    = 7.0834907*SVector(cosd(ϕ), sind(ϕ))

2-element SVector{2, Float64} with indices SOneTo(2):
  0.0
 -7.0834907

In [56]:
f(t::Float64) = (1+sin(5t)) .* (cos(t), sin(t))
g(t::Float64, a::Float64) = (a+sin(5*t - pi/4)) .* (cos(t), sin(t))
θ = LinRange(-pi, pi, N)


400-element LinRange{Float64, Int64}:
 -3.14159, -3.12585, -3.1101, -3.09435, …, 3.09435, 3.1101, 3.12585, 3.14159

Create boundary

In [57]:
z = g.(θ,2.0)
x, y            = first.(z), last.(z)
x, y            = GeometryUtils.divideCurve(x, y, N)
xm, ym          = GeometryUtils.calcMidpoints(x,y)
arc_lengths     = GeometryUtils.calcArcLength(x,y)
distance_matrix = GeometryUtils.calcDistances(xm,ym)
;


Define a domain

In [58]:
xdom = LinRange(-6,6, NDOM)
ydom = LinRange(-6,6, NDOM)
GRID = RectilinearGrid(xdom, ydom)
MESH = SimpleMesh(vertices(GRID), GRID.topology)
COORDS = coordinates.(vertices(MESH))

XDOM, YDOM = first.(COORDS), last.(COORDS)
;

Perform BWM

In [59]:
wave1 = BoundaryWall.boundaryWallWave(K, 
                                      xm, 
                                      ym,
                                      XDOM,
                                      YDOM,
                                      SIGMA,
                                      distance_matrix,
                                      arc_lengths,
                                      length(arc_lengths),
                                      diagind(size(distance_matrix)...))
;


In [60]:
let 
  fig = Figure()
  ax = [Axis(fig[1,j],xtickalign=1,ytickalign=1,xticksmirrored=1,yticksmirrored=1,xgridvisible=false, ygridvisible=false) for j in 1:2]
  # heatmap!(ax, xdom[:], ydom[:], real(wave), colormap=:dense)
  # scatter!(ax, XDOM, YDOM, color=real)
  viz!(ax[1], MESH, color=abs2.(wave1), colorscheme=:dense)
  viz!(ax[2], MESH, color=angle.(wave1), colorscheme=:twilight)
  
  [ax.xticks=minimum(XDOM):2.0:maximum(XDOM) for ax in ax]
  [ax.yticks=minimum(YDOM):2.0:maximum(YDOM) for ax in ax]

  [lines!(ax, x, y, color=:black) for ax in ax]
  [ax.aspect=DataAspect() for ax in ax]
  fig
end